In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        continue
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import joblib
from sklearn.preprocessing import MinMaxScaler

torch.manual_seed(0)
np.random.seed(0)
calculate_loss_over_all_values = False

In [2]:
# # for TPU
# device = xm.xla_device()
# torch.set_default_tensor_type('torch.FloatTensor')
# import torch_xla
# import torch_xla.core.xla_model as xm

# # FOR GPU AND CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number
#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)
#print(out)

input_window = 300
output_window = 5
batch_size = 20 # batch size

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransAm(nn.Module):
    def __init__(self,feature_size=30,num_layers=2,dropout=0.2):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()
    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data2(inst):
    this_inst_df = joblib.load(f'./dt11112222/{inst}.p')
    amplitude = this_inst_df['close'].to_numpy()
    amplitude = amplitude.reshape(-1)
    print(amplitude.shape)
    
    scaler = MinMaxScaler(feature_range=(-15, 15)) 
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    sampels = int(amplitude.shape[0]*0.95)
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window]
    print(train_sequence.shape)
    
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window]
    print(test_data.shape)
    
    return train_sequence.to(device),test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        
        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
plot_counter = 0
def plot_and_loss(eval_model, data_source,epoch,tknip):
    global plot_counter
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()

            test_result = torch.cat((test_result.to(device), output[-1].view(-1).to(device)), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth.to(device), target[-1].view(-1).to(device)), 0)
            
#             test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
#             truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    test_result = test_result.cpu().numpy()
    truth = truth.cpu().numpy()
    len(test_result)
    plot_counter +=1
    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(f'./plots_5x/transformer-epoch_{plot_counter}_{epoch}_{tknip}.png')
   
    pyplot.close()
    
    return total_loss / i


def predict_future(eval_model, data_source,steps,tknip):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
#     data = data.to(device).view(-1)
    data = data.cpu().view(-1)
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(f'./plots_5x/transformer-future_{plot_counter}_{steps}_{tknip}.png')
    pyplot.close()
        
# entweder ist hier ein fehler im loss oder in der train methode, aber die ergebnisse sind unterschiedlich 
# auch zu denen der predict_future
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).to(device).item()
#                 total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).to(device).item()            
#                 total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

In [4]:
# model = TransAm().to(device)
model= torch.load('./best_model_multi11.pt')

valid_loss_list = []

criterion = nn.MSELoss()
lr = 0.000001
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 1 # The number of epochs
best_model = None

In [ ]:
counter = 0
files_to_consider = 15

load_counter = 0

while True:
    for dirname, _, filenames in os.walk('./dt11112222/'):
        for filename in filenames:
            path_str = os.path.join(dirname, filename)
            split_lst = path_str.split('.')
            contains = 'p' in split_lst
            if(contains): ## and counter<files_to_consider

                one_plty = filename.split('/')[0].split('.')[0]

                train_data, val_data = get_data2(one_plty)

                for epoch in range(1, epochs + 1):
                    epoch_start_time = time.time()
                    train(train_data)

                    if(epoch % 1 == 0):
                        val_loss = plot_and_loss(model, val_data,epoch,one_plty)
                        predict_future(model, val_data,200,one_plty)
                    else:
                        val_loss = evaluate(model, val_data)

                    print('-' * 89)
                    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                                     val_loss, math.exp(val_loss)))
                    print('-' * 89)
                    
                    torch.save(model,'./best_model_multi10.pt')
                    torch.save(model,f'./inst_wise_model_5x/{one_plty}.pt')
                    
                    load_counter+=1
                    valid_loss_list.append({'inst':one_plty,
                                            'loss':val_loss})
                    valid_loss_map_df = pd.DataFrame(valid_loss_list)
                    valid_loss_map_df.to_excel('./valid_loss_map_df_5x.xlsx')
                    
                    
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        best_model = model
                        torch.save(best_model,'./best_model_multi10_best.pt')
                        model= torch.load('./best_model_multi11_best.pt')

    

                #src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
                #out = model(src)
                #
                #print(out)
                #print(out.shape)

                print(path_str)
                counter+=1
                
    scheduler.step() 

(115880,)
torch.Size([109781, 2, 300])
torch.Size([5489, 2, 300])


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |  1097/ 5489 batches | lr 0.000001 | 61.29 ms | loss 0.53168 | ppl     1.70
| epoch   1 |  2194/ 5489 batches | lr 0.000001 | 61.15 ms | loss 0.04943 | ppl     1.05
| epoch   1 |  3291/ 5489 batches | lr 0.000001 | 61.09 ms | loss 0.04382 | ppl     1.04
| epoch   1 |  4388/ 5489 batches | lr 0.000001 | 61.11 ms | loss 0.09877 | ppl     1.10
| epoch   1 |  5485/ 5489 batches | lr 0.000001 | 61.25 ms | loss 0.11499 | ppl     1.12
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 353.26s | valid loss 10.87611 | valid ppl 52897.26
-----------------------------------------------------------------------------------------
./dt11112222/2952193.p
(115877,)
torch.Size([109778, 2, 300])
torch.Size([5489, 2, 300])


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |  1097/ 5488 batches | lr 0.000001 | 61.24 ms | loss 2.63398 | ppl    13.93
| epoch   1 |  2194/ 5488 batches | lr 0.000001 | 57.01 ms | loss 0.35737 | ppl     1.43
| epoch   1 |  3291/ 5488 batches | lr 0.000001 | 53.35 ms | loss 0.07555 | ppl     1.08
| epoch   1 |  4388/ 5488 batches | lr 0.000001 | 61.31 ms | loss 0.18780 | ppl     1.21
| epoch   1 |  5485/ 5488 batches | lr 0.000001 | 61.25 ms | loss 0.65155 | ppl     1.92
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 340.90s | valid loss 2.02218 | valid ppl     7.55
-----------------------------------------------------------------------------------------
./dt11112222/895745.p
(115858,)
torch.Size([109760, 2, 300])
torch.Size([5488, 2, 300])


...

| epoch   1 |  1097/ 5487 batches | lr 0.000001 | 51.42 ms | loss 0.09378 | ppl     1.10
| epoch   1 |  2194/ 5487 batches | lr 0.000001 | 61.17 ms | loss 0.08141 | ppl     1.08
| epoch   1 |  3291/ 5487 batches | lr 0.000001 | 61.19 ms | loss 2.90848 | ppl    18.33
| epoch   1 |  4388/ 5487 batches | lr 0.000001 | 61.12 ms | loss 0.16932 | ppl     1.18
| epoch   1 |  5485/ 5487 batches | lr 0.000001 | 61.15 ms | loss 0.58217 | ppl     1.79
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 342.36s | valid loss 0.80519 | valid ppl     2.24
-----------------------------------------------------------------------------------------
./dt11112222/54273.p
(29306,)
torch.Size([27535, 2, 300])
torch.Size([1161, 2, 300])


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   275/ 1376 batches | lr 0.000001 | 61.51 ms | loss 0.30114 | ppl     1.35
| epoch   1 |   550/ 1376 batches | lr 0.000001 | 61.24 ms | loss 2.00446 | ppl     7.42
| epoch   1 |   825/ 1376 batches | lr 0.000001 | 61.15 ms | loss 0.14452 | ppl     1.16
| epoch   1 |  1100/ 1376 batches | lr 0.000001 | 61.11 ms | loss 0.34151 | ppl     1.41
| epoch   1 |  1375/ 1376 batches | lr 0.000001 | 61.20 ms | loss 3.33897 | ppl    28.19
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 88.35s | valid loss 4.01719 | valid ppl    55.54
-----------------------------------------------------------------------------------------
./dt11112222/3484417.p
(115849,)
torch.Size([109751, 2, 300])
torch.Size([5488, 2, 300])


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |  1097/ 5487 batches | lr 0.000001 | 61.47 ms | loss 0.39130 | ppl     1.48
| epoch   1 |  2194/ 5487 batches | lr 0.000001 | 51.40 ms | loss 0.65365 | ppl     1.92
| epoch   1 |  3291/ 5487 batches | lr 0.000001 | 61.16 ms | loss 2.54352 | ppl    12.72
| epoch   1 |  4388/ 5487 batches | lr 0.000001 | 61.14 ms | loss 0.86538 | ppl     2.38
| epoch   1 |  5485/ 5487 batches | lr 0.000001 | 61.18 ms | loss 0.23546 | ppl     1.27
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 342.61s | valid loss 11.21307 | valid ppl 74092.71
-----------------------------------------------------------------------------------------
./dt11112222/40193.p
(115839,)
torch.Size([109742, 2, 300])
torch.Size([5487, 2, 300])


In [ ]:
# for epoch in range(1, epochs + 1):
#     epoch_start_time = time.time()
#     train(train_data)
    
#     if(epoch % 1 == 0):
#         val_loss = plot_and_loss(model, val_data,epoch)
#         predict_future(model, val_data,200)
#     else:
#         val_loss = evaluate(model, val_data)
        
#     print('-' * 89)
#     print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
#                                      val_loss, math.exp(val_loss)))
#     print('-' * 89)

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_model = model
#         torch.save(best_model,'./best_model_multi.pt')

#     scheduler.step() 

# #src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
# #out = model(src)
# #
# #print(out)
# #print(out.shape)

In [ ]:
# src = torch.rand(30, batch_size, 30) # (source sequence length,batch size,feature number) 
# src = src.to(device="cuda")
# # out = gf(val_data[0])
# print(src.shape)
# out = model(src)

# print(out)
# print(out.shape)

In [ ]:
# gf= torch.load('../input/dt11112222/best_model_multi.pt')